In [ ]:
#Importing libraries
import requests
from bs4 import BeautifulSoup as soup
import csv
import pandas as pd
import os
import regex as re
from openpyxl import load_workbook

In [ ]:
url = "https://www.coingecko.com/" #The site from which the data will be gathered
top_100_crypto_links = []
response = requests.get(url)
sriuba = soup(response.content, 'html.parser') #BeautifulSoup instance of the site
#Gathering the links to individual currencies
for x in sriuba.find('div', {'class' : 'coingecko-table'}).find('tbody').find_all('tr'):
    for y in x.find_all('a', class_ = 'tw-hidden lg:tw-flex font-bold tw-items-center tw-justify-between'):
        top_100_crypto_links.append(url + str(y.get('href')) + '/historical_data/eur#panel')
#Creating an excel file for storing the data
FilePath = os.getcwd() + '\\Crypto_prices.xlsx'
FilePath = FilePath.replace('\\', '/')
pd.DataFrame().to_excel('Crypto_prices.xlsx')
ExcelWorkbook = load_workbook(FilePath)
writer = pd.ExcelWriter(FilePath, engine = 'openpyxl')
writer.book = ExcelWorkbook
#Gathering data from individual links of the crypto currencies
psl = 0
for link in top_100_crypto_links:
    response = requests.get(link)
    sriuba = soup(response.content, 'html.parser')
    lst = []
    for name in sriuba.find_all('div', class_ = 'mr-md-3 tw-pl-2 md:tw-mb-0 tw-text-xl tw-font-bold tw-mb-0'): #Finding the name of the currency
        psl += 1
        print('Sheet{}: '.format(int(psl)) + name.text)
    for x in sriuba.find('tbody'): #Gathering the data, storing it to a dataframe and then appending it to the excel file
        if type(x.find('th')) != int:
            lst.append(x.text.split())
            dataframe = pd.DataFrame(lst, columns = ['Date', 'Market Cap', 'Volume', 'Open', 'Close']).to_excel(writer, sheet_name = str(name.text))
            writer.save()
            writer.close()